# Preparação do ambiente

## Bibliotecas

In [1]:
import configparser
import hashlib
import json
import os
import pandas as pd
import requests
import time
import urllib.parse

from pathlib import Path
from tqdm.auto import tqdm

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'

## Constantes e funções auxiliares

In [3]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'

In [4]:
def get_new_access_token(actual_refresh_token,client_id,client_secret):
    url = 'https://api.mercadolibre.com/oauth/token'
    payload = {
        'grant_type': 'refresh_token', 
        'client_id': client_id,
        'client_secret': client_secret, 
        'refresh_token': actual_refresh_token
    }
        
    headers = {
        'accept': 'application/json',
        'content-type': 'application/x-www-form-urlencoded'
    }
        
    response = requests.post(url, headers=headers, data=payload)
    content = response.content.decode(response.encoding)
    content =  json.loads(content)
        
    return content['access_token'], content['refresh_token']

In [ ]:
# def load_sch(sch_database_file):
    
#     # load SCH database
#     usecols = [0,1,11,12,13,14,15]
#     dtype = {'Número de Homologação': 'str'}
#     parse_dates = [0]
#     date_format = '%d/%m/%Y'

#     df_sch = pd.read_csv(
#         sch_database_file,
#         sep=';',
#         usecols=usecols,
#         dtype=dtype,
#         parse_dates=parse_dates,
#         date_format=date_format
#         )

#     df_sch = df_sch.sort_values(by='Data da Homologação',ascending=False)

#     df_modelo = df_sch[['Número de Homologação','Modelo']].dropna()
#     df_modelo = df_modelo.groupby('Número de Homologação',as_index=False)['Modelo'].apply(lambda x: ' | '.join(x))
#     df_modelo = df_modelo.drop_duplicates(subset='Número de Homologação')

#     df_nome_comercial = df_sch[['Número de Homologação','Nome Comercial']].dropna()
#     df_nome_comercial = df_nome_comercial.groupby('Número de Homologação',as_index=False)['Nome Comercial'].apply(lambda x: ' | '.join(x))
#     df_nome_comercial = df_nome_comercial.drop_duplicates(subset='Número de Homologação')

#     columns_to_keep = ['Data da Homologação', 'Número de Homologação', 'Nome do Fabricante']
#     df_sch_models = df_sch[columns_to_keep]
#     df_sch_models = df_sch_models.drop_duplicates(subset='Número de Homologação')

#     df_sch_models = df_sch_models.merge(df_modelo,how='left')
#     df_sch_models = df_sch_models.merge(df_nome_comercial,how='left')
#     df_sch_models = df_sch.fillna('')
#     df_sch_models['Modelo Completo'] = df_sch_models[['Modelo', 'Nome Comercial']].apply(lambda row: ' | '.join(row) if len(row['Nome Comercial'])>0 else row['Modelo'],axis=1)

#     columns_to_keep = ['Número de Homologação', 'Nome do Fabricante', 'Modelo Completo']
#     df_sch_models = df_sch_models.drop_duplicates(subset='Número de Homologação')
#     df_sch_models = df_sch_models[columns_to_keep].reset_index(drop=True)
    
#     return df_sch, df_sch_models

In [9]:
def parse_item(item_result):
    keys_to_keep = ['id', 'title', 'condition', 'permalink', 'category_id', 'domain_id']
    parsed_item = {key: item_result[key] for key in keys_to_keep}
    
    attributes_to_keep = ['BRAND', 'MODEL']
    for attribute in item_result['attributes']:
        if attribute['id'] in attributes_to_keep:
            parsed_item[attribute['id'].lower()] = attribute['value_name']
    return parsed_item

In [10]:
def parse_item_details(result, return_anatel_homologation_number=False):
    
    keys_to_keep = ['id', 'title', 'catalog_product_id', 'permalink', 'category_id', 'domain_id', 
                    'currency_id', 'price', 'original_price', 
                    'initial_quantity', 'available_quantity', 'official_store_id', 'official_store_name']
    
    parsed_result = {}
    for key in keys_to_keep:
        parsed_result[key] = result.get(key)
    
    if result.get('seller') is not None:
        parsed_result['seller_id'] = result.get('seller').get('id')
        parsed_result['seller_nickname'] = result.get('seller').get('nickname')
    else:
        parsed_result['seller_id'] = None
        parsed_result['seller_nickname'] = None    

    brand = None
    model = None
    gtin = None
    detailed_model = None
    anatel_homologation_number = None
    cellphones_anatel_homologation_number = None
    
    for attribute in result['attributes']:
        if attribute['id'] == 'BRAND':
            brand = attribute['value_name']
        elif attribute['id'] == 'MODEL':
            model = attribute['value_name']
        elif attribute['id'] == 'DETAILED_MODEL':
            detailed_model = attribute['value_name'] 
        elif attribute['id'] == 'GTIN':
            gtin = attribute['value_name'] 
        elif attribute['id'] == 'ANATEL_HOMOLOGATION_NUMBER':
            anatel_homologation_number = attribute['value_name']
        elif attribute['id'] == 'CELLPHONES_ANATEL_HOMOLOGATION_NUMBER':
            cellphones_anatel_homologation_number = attribute['value_name']

    if return_anatel_homologation_number:
        if anatel_homologation_number is None:
            item_id = parsed_result['id']
            attr_url = f'https://api.mercadolibre.com/items/{item_id}?attributes=attributes&include_internal_attributes=true'
            headers = {'user-agent': USER_AGENT}
            attr_response = requests.get(attr_url,headers=headers)
            attr_content = attr_response.content.decode(attr_response.encoding)
            attr_content =  json.loads(attr_content)
            attr_content = attr_content['attributes']
            for attr in attr_content:
                if attr['id'] == 'ANATEL_HOMOLOGATION_NUMBER':
                    anatel_homologation_number = attr['value_name']
    
        if cellphones_anatel_homologation_number is None:
            item_id = parsed_result['id']
            attr_url = f'https://api.mercadolibre.com/items/{item_id}?attributes=attributes&include_internal_attributes=true'
            headers = {'user-agent': USER_AGENT}
            attr_response = requests.get(attr_url,headers=headers)
            attr_content = attr_response.content.decode(attr_response.encoding)
            attr_content =  json.loads(attr_content)
            attr_content = attr_content['attributes']
            for attr in attr_content:
                if attr['id'] == 'CELLPHONES_ANATEL_HOMOLOGATION_NUMBER':
                    cellphones_anatel_homologation_number = attr['value_name']
    else:
        anatel_homologation_number = '999999999999'
        cellphones_anatel_homologation_number = '999999999999'

    parsed_result['brand'] = brand
    parsed_result['model'] = model
    parsed_result['detailed_model'] = detailed_model
    parsed_result['gtin'] = gtin
    parsed_result['anatel_homologation_number'] = anatel_homologation_number
    parsed_result['cellphones_anatel_homologation_number'] = cellphones_anatel_homologation_number
    
    return parsed_result

# Carga e prepação dos dados

In [5]:
creds_file = Path(os.environ['USERPROFILE'],'creds.ini')
creds = configparser.ConfigParser()
creds.read(creds_file)  

client_id = creds['MERCADO_LIVRE']['client_id']
client_secret = creds['MERCADO_LIVRE']['client_secret']
actual_refresh_token = creds['MERCADO_LIVRE']['actual_refresh_token']
actual_access_token = creds['MERCADO_LIVRE']['actual_access_token']

actual_access_token, actual_refresh_token = get_new_access_token(actual_refresh_token,client_id,client_secret)
creds['MERCADO_LIVRE']['actual_access_token'] = actual_access_token
creds['MERCADO_LIVRE']['actual_refresh_token'] = actual_refresh_token
    
with open(creds_file, 'w') as file:
    creds.write(file)

# Pesquisa

## Pesquisa categoria *"Celulares e Smartphones"*

Pesquisa todos os produtos da categoria *"Celulares e Smartphones"*. A API pública do Mercado Livre limita a busca a 4000 itens. A partir dos resultados dessa pesquisa serão obtidos os modelos dos celulares encontrados (Marca+Modelo) para realizar novas buscas, uma para cada modelos.

A busca por modelo também é limitada, permite buscar apenas 1000 itens, mas, com várias buscas, poderão ser encontrados mais itens do que os 4000 iniciais da busca por categoria.

In [7]:
# Get first page of results
url = 'https://api.mercadolibre.com/sites/MLB/search?category=MLB1055'
params = {'offset': 0}
headers = {
  'Authorization': f'Bearer {actual_access_token}'
}

response = requests.request("GET", url, headers=headers, params=params)
# response = requests.get(url, params=params)
content = response.content.decode(response.encoding)
content =  json.loads(content)
category_results = content['results']

total_items = content['paging']['total']
next_offsets = list(range(50,total_items+1,50))

print(f'Total items found: {total_items}')

for offset in tqdm(next_offsets):
    
    params = {'offset': offset}
    response = requests.request("GET", url, headers=headers, params=params)   
    # response = requests.get(url, params=params)
    
    if response.status_code == 200:
        content = response.content.decode(response.encoding)
        content =  json.loads(content)
        category_results.extend(content['results'])
    else:
        break

print('Total items collected:', len(category_results))

Total items found: 63778


  0%|          | 0/1275 [00:00<?, ?it/s]

Total items collected: 3885


In [11]:
df_category_items = pd.DataFrame([parse_item(item) for item in category_results])

df_brand_models = df_category_items[['brand', 'model']].drop_duplicates()
models_to_seach = df_brand_models.apply(lambda row: ' '.join(row),axis=1).to_list()
models_to_seach[:5]

['Samsung M35 5G',
 'Samsung A15 4G',
 'Samsung A15 5G Dual Sim',
 'Motorola G54 5G',
 'Motorola G04S']

## Pesquisa por modelos

In [19]:
brand_model_results = []

for model in tqdm(models_to_seach):

    query = urllib.parse.quote_plus(model)
    
    url = 'https://api.mercadolibre.com/sites/MLB/search'
    params={'q': query, 'offset': 0, 'category': 'MLB1055'}
    headers = {'user-agent': USER_AGENT}
    
    response = requests.get(url,params=params,headers=headers)
    content = response.content.decode(response.encoding)
    content =  json.loads(content)
    brand_model_results.extend(content['results'])
    
    total_items = content['paging']['total']
    next_offsets = list(range(50,total_items+1,50))

    # Max offset for public API is 1000
    for offset in tqdm(next_offsets[:20],leave=False):
        params={'q': query, 'offset': offset, 'category': 'MLB1055'}
        response = requests.get(url,params=params,headers=headers) 
        
        if response.status_code == 200:
            content = response.content.decode(response.encoding)
            content =  json.loads(content)
            brand_model_results.extend(content['results'])
        else:
            break

df_brand_models = pd.DataFrame([parse_item(result) for result in brand_model_results])
df_brand_models = df_brand_models[(df_brand_models['condition']=='new')].drop_duplicates(subset='id')
df_brand_models

  0%|          | 0/1851 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

,id,title,condition,permalink,category_id,domain_id,brand,model
0,MLB3709441027,"Celular Samsung Galaxy M35 5g , Câmera Tripla ...",new,https://www.mercadolivre.com.br/celular-samsun...,MLB1055,MLB-CELLPHONES,Samsung,M35 5G
1,MLB3709441025,"Celular Samsung Galaxy M35 5g , Câmera Tripla ...",new,https://www.mercadolivre.com.br/celular-samsun...,MLB1055,MLB-CELLPHONES,Samsung,M35 5G
2,MLB4720524452,"Celular Samsung Galaxy M35 5g , Câmera Tripla ...",new,https://www.mercadolivre.com.br/celular-samsun...,MLB1055,MLB-CELLPHONES,Samsung,M35 5G
5,MLB4167170704,"Smartphone Samsung Galaxy M34 5g, 128gb, 6gb, ...",new,https://www.mercadolivre.com.br/smartphone-sam...,MLB1055,MLB-CELLPHONES,Samsung,M34
6,MLB4474612432,Samsung Galaxy A25 5g Dual Sim 256 Gb Azul 8 G...,new,https://www.mercadolivre.com.br/samsung-galaxy...,MLB1055,MLB-CELLPHONES,Samsung,A25 5G Dual SIM
...,...,...,...,...,...,...,...,...
311836,MLB2791967049,Samsung Galaxy M31 128gb Azul Muito Bom - Troc...,new,https://produto.mercadolivre.com.br/MLB-279196...,MLB1055,MLB-CELLPHONES,Samsung,Galaxy M31
311838,MLB2794342319,Samsung Galaxy M31 128gb Azul Bom - Trocafone-...,new,https://produto.mercadolivre.com.br/MLB-279434...,MLB1055,MLB-CELLPHONES,Samsung,Galaxy M31
311839,MLB2794372202,Celular Samsung Galaxy M31 128gb Azul Muito Bo...,new,https://produto.mercadolivre.com.br/MLB-279437...,MLB1055,MLB-CELLPHONES,Samsung,Galaxy M31
311841,MLB2636642319,Samsung Galaxy M31 128gb Rosa Bom - Usado,new,https://produto.mercadolivre.com.br/MLB-263664...,MLB1055,MLB-CELLPHONES,Samsung,Galaxy M31


In [21]:
df_brand_models.to_parquet('brand_models.parquet')

In [20]:
df_brand_models.brand.value_counts()

brand
Xiaomi                5745
Samsung               4244
Apple                 2668
Motorola              2017
Multilaser            1538
                      ... 
TLC                      1
Red                      1
Redmi 12 8 RAM/256       1
Pova 5                   1
Apark                    1
Name: count, Length: 363, dtype: int64

In [13]:
df_brand_models.category_id.value_counts()

category_id
MLB1055      1602
MLB270375       1
MLB454750       1
Name: count, dtype: int64

In [ ]:
with open('brand_model_results.json', 'w') as f:
    json.dump(brand_model_results,f,indent=2)

## Consolidação e análise

In [ ]:
df_cellphones = pd.DataFrame(parsed_results)
df_cellphones.to_parquet('cellphone.parquet')

columns_to_keep = ['id', 'permalink', 'title','seller_nickname', 'brand', 'model', 'gtin', 'cellphones_anatel_homologation_number']
df_analise = df_cellphones[columns_to_keep]

columns_to_keep = ['id', 'permalink', 'title', 'seller_nickname', 'brand', 'model', 'ean_anuncio', 'sch_anuncio']
df_analise.columns = columns_to_keep

df_analise['ean_anuncio'] = df_analise['ean_anuncio'].str.split(',')
df_analise = df_analise.explode('ean_anuncio')

# df_analise['ean_presente'] = df_analise['ean_mp'].apply(lambda x:  0 if x is None else 1)
df_analise['sch_presente'] = df_analise['sch_anuncio'].apply(lambda x:  0 if x is None else 1)

df_analise['sch_anuncio'] = df_analise['sch_anuncio'].fillna('0').str.zfill(12)
df_analise['ean_anuncio'] = df_analise['ean_anuncio'].fillna('0').str.zfill(13)

df_analise = df_analise.merge(df_ean_sch,left_on='ean_anuncio', right_on='ean_sch', how='left')
df_analise['ean_ok'] = df_analise['ean_anuncio']==df_analise['ean_sch']
df_analise['sch_ok'] = df_analise['sch_anuncio']==df_analise['sch_sch']
df_analise['ean_sch_ok'] = df_analise[['ean_ok', 'sch_ok']].apply(lambda row: all(row),axis=1)
columns_to_keep = ['id', 'permalink', 'title', 'seller_nickname', 'brand', 'model', 'ean_anuncio', 'sch_anuncio', 'sch_presente', 'ean_sch', 'sch_sch', 'ean_sch_ok']
df_analise = df_analise[columns_to_keep]


df_analise = df_analise.merge(df_sch_models, left_on='sch_anuncio', right_on='Número de Homologação',how='left')
df_analise['sch_valido'] = df_analise['Número de Homologação'].apply(lambda x:  0 if pd.isna(x) else 1)
columns_to_keep = ['id', 'permalink', 'title', 'seller_nickname', 'brand', 'model', 'ean_anuncio', 'sch_anuncio', 'sch_presente', 'ean_sch', 'sch_sch', 'ean_sch_ok', 'sch_valido', 'Nome do Fabricante', 'Modelo Completo']
df_analise = df_analise[columns_to_keep]

df_analise

In [ ]:
df_analise[df_analise['ean_anuncio']=='0000000000000'].shape

In [ ]:
columns_to_group = ['id', 'sch_presente', 'sch_valido', 'ean_sch_ok']

agg_func = {'id': 'count', 'sch_presente': 'sum', 'sch_valido': 'sum', 'ean_sch_ok': 'sum'}
df_resumo = pd.DataFrame(df_analise[columns_to_group].agg(agg_func)).T

df_resumo['pe_sch_presente'] = df_resumo['sch_presente']/df_resumo['id']*100
df_resumo['pe_sch_valido'] = df_resumo['sch_valido']/df_resumo['id']*100
df_resumo['pe_ean_sch_ok'] = df_resumo['ean_sch_ok']/df_resumo['id']*100

columns_to_keep = ['Total de anúncios', 'Anúncios com código SCH Informado', 'Anúncios com código SCH Válido', 'Anúncios com par EAN/SCH Válido',
                   '% Anúncios com código SCH Informado', '% Anúncios com código SCH Válido', '% Anúncios com par EAN/SCH Válido']

df_resumo.columns = columns_to_keep
df_resumo.index.name = 'Mercado Livre'

df_resumo

In [ ]:
columns_to_group = ['seller_nickname', 'id', 'sch_presente', 'sch_valido', 'ean_sch_ok']
agg_func = {'id': 'count', 'sch_presente': 'sum', 'sch_valido': 'sum', 'ean_sch_ok': 'sum'}
df_seller = df_analise[columns_to_group].groupby(columns_to_group[0]).agg(agg_func)

df_seller['pe_sch_presente'] = df_seller['sch_presente']/df_seller['id']*100
df_seller['pe_sch_valido'] = df_seller['sch_valido']/df_seller['id']*100
df_seller['pe_ean_sch_ok'] = df_seller['ean_sch_ok']/df_seller['id']*100

columns_to_keep = ['Total de anúncios', 'Anúncios com EAN Informado', 'Anúncios com código SCH Informado', 'Anúncios com código SCH Válido', 
                   '% Anúncios com EAN Informado', '% Anúncios com código SCH Informado', '% Anúncios com código SCH Válido']

df_seller.columns = columns_to_keep
df_seller.index.name = 'Vendedor'

df_seller.sort_values(by='Total de anúncios',ascending=False).head(10)

In [ ]:
columns_to_group = ['brand', 'id', 'sch_presente', 'sch_valido', 'ean_sch_ok']
agg_func = {'id': 'count', 'sch_presente': 'sum', 'sch_valido': 'sum', 'ean_sch_ok': 'sum'}
df_brand = df_analise[columns_to_group].groupby(columns_to_group[0]).agg(agg_func)

df_brand['pe_sch_presente'] = df_brand['sch_presente']/df_brand['id']*100
df_brand['pe_sch_valido'] = df_brand['sch_valido']/df_brand['id']*100
df_brand['pe_ean_sch_ok'] = df_brand['ean_sch_ok']/df_brand['id']*100

columns_to_keep = ['Total de anúncios', 'Anúncios com EAN Informado', 'Anúncios com código SCH Informado', 'Anúncios com código SCH Válido', 
                   '% Anúncios com EAN Informado', '% Anúncios com código SCH Informado', '% Anúncios com código SCH Válido']

df_brand.columns = columns_to_keep
df_brand.index.name = 'Fabricante'

df_brand.sort_values(by='Total de anúncios',ascending=False).head(10)